# ASSIGNMENT 1: KNN

Algorithm which can be used for both classification as well as regression predictive problems. K-nearest neighbors (KNN) algorithm uses ‘feature similarity’ to predict the values of new datapoints which further means that the new data point will be assigned a value based on how closely it matches the points in the training set.

In this assignment, it is asked to implement KNN algorithm from the scratch and to apply this algorithm for both given classification and regression problems. This report shows steps we have followed, code implementations and analysis of results for KNN algorithms.

### Common Code Implementations

Some functions are used in both parts, thus, we prefer to gather them below this section in order to avoid unnecessary repetition in report.

In [8]:
from copy import deepcopy
from functools import reduce
from math import sqrt
import pandas

In [2]:
#converts given .xls file to pandas dataframe
def read_csv_file(filename):
    return pandas.read_csv(filename)

In [3]:
# creates new normalized values by maintaining data distribution and scale
# normalization is done according to min and max values in that particular column
def normalize_dataset(dataset, target):
    normalized_dataset = dataset.copy()
    for column in list(dataset.columns.values):
        if column != target:
            normalized_dataset[column] = (normalized_dataset[column] - normalized_dataset[column].min()) / (
                    normalized_dataset[column].max() - normalized_dataset[column].min())
    return normalized_dataset

For machine learning, normalization is a data preparation approach in order to make the most accurate predcition. It is aimed that by normalizing the data, new values will be created that maintain the source data's basic distribution and ratios while avoiding difficulties that may arise during modeling. Instead of absolute numbers, normalized data are represented as percentile ranks in the range between 0 to 1. 

In [24]:
# splits dataframe into n folds
def cross_validation_splitter(data, n_folds):
    dataframe_split = list()
    # shuffle data
    shuffle_data = data.sample(frac=1)
    for i in range(n_folds):
        dataframe_split.insert(i, fold_i_of_k(shuffle_data, i + 1, n_folds))
    return dataframe_split

In [25]:
# return the ith fold
def fold_i_of_k(dataset, i, k):
    n = len(dataset)
    return dataset[n * (i - 1) // k:n * i // k]

K-fold cross validation is where a given data set is split into a K number of folds where each fold is used as a testing set at some point. In our assignment, we take the scenario of 5-Fold cross validation(K=5). Here, the data set is split into 5 folds. In the first iteration, the first fold is used to test the model and the rest are used to train the model. In the second iteration, 2nd fold is used as the testing set while the rest serve as the training set. This process is repeated until each fold of the 5 folds have been used as the testing set.

In [13]:
# calculates euclidean distance between two rows according to formula
def calculate_euclidean_distance(row1, row2):
    distance = 0.0
    # iterate over each column, calculate the distance, add it to total distance
    for i in range(len(row1[1]) - 1):
        distance += (row1[1][i] - row2[1][i]) ** 2
    return sqrt(distance)

In [15]:
# returns the most occurred type value in the nearest neighbors
def predict_type(nearest_neighbors_types):
    return max(set(nearest_neighbors_types), key=nearest_neighbors_types.count)

### PART 1: Glass Material Classification

In this part, nearest neighbor algorithm is used in order to classify glass types of different glass

In [22]:
# this function is the main function for classification
def classification():
    # declares the main variables used across the functions
    num_neighbors = [1, 3, 5, 7, 9]
    k_folds = 5
    data = read_csv_file("glass.csv.xls")
    target = "Type"
    normalized_data = normalize_dataset(data, target)

    # invokes KNN classification function by changing data and algorithm types
    
    # KNN classification without feature normalization
    for num in num_neighbors:
        scores = knn_classification(data, k_folds, num, "normal", target)
        
    # KNN classification with feature normalization
    for num in num_neighbors:
        scores = knn_classification(normalized_data, k_folds, num, "normal", target)

    # weighted KNN classification without feature normalization
    for num in num_neighbors:
        scores = knn_classification(data, k_folds, num, "weighted", target)
        
    # KNN classification with feature normalization
    for num in num_neighbors:
        scores = knn_classification(normalized_data, k_folds, num, "weighted", target)

In [19]:
# cross validation, type predicition and accuracy calculation are done here
def knn_classification(dataset, n_folds, k_neighbors, algo_type, target):
    # create 5 folds from dataset
    folds = cross_validation_splitter(dataset, n_folds)
    results = list()

    # create test and train sets
    for i in range(len(folds)):
        # deepcopy data so that the original dataframe does not change
        train_set = deepcopy(folds) 
        # choose ith fold as test set and remove it from train data
        test_set = train_set.pop(i)
        # original -> keeps original types for accuracy metric comparison
        original = list()

        for row in folds[i].iterrows():
            original.append(row[1][target])

        predictions = list()
        # call the related function(weighted/non-weighted) for each test data with current train set
        # return predicted types by the written algorithm
        for row in test_set.iterrows():
            if algo_type == "normal":
                output = predict_type(get_nearest_neighbors(train_set, row, k_neighbors))
                predictions.append(output)
            elif algo_type == "weighted":
                output = predict_weighted_type(train_set, row, k_neighbors)
                predictions.append(output)

        # calculate accuracy metric with original and predicted types
        accuracy = accuracy_metric(original, predictions)
        results.append(accuracy)

    return results

In this function, first of all we split the data into 5-folds. Then, we iterate over the folds and in each iteration, we choose ith fold as test data and the remaining ones as the train data. For each test row, we get the nearest neighbors and predict its type. Lastly, we compare the predicted and actual values and calculate the accuracy of our algorithm.

In [12]:
# find the nearest neighbors of the test row
def get_nearest_neighbors(train_data, test_row, k):
    # distances are kept as dictionary because we need both type and distance 
    # in order to sort distance and return the nearest ones' types
    distances = dict()

    for train in train_data:
        for train_row in train.iterrows():
            #calculate distance for each train row and test row
            distances[calculate_euclidean_distance(train_row, test_row)] = train_row[1]["Type"]
    # sort the dictionary values according to distances -> keys and get first k ones
    sorted_distances = sorted(distances.items(), key=lambda x: x[0])[:k]
    # return only the types
    return [item[1] for item in sorted_distances]

In [35]:
def predict_weighted_type(train_data, test_row, k):
    distances = dict()

    # finds nearest neighbors
    for train in train_data:
        for train_row in train.iterrows():
            distances[calculate_euclidean_distance(train_row, test_row)] = train_row[1]["Type"]
    
    # get the first k neighbors
    sorted_distances = sorted(distances.items(), key=lambda x: x[0])[:k]

    # calculates weighted distances from neighbors
    weight = dict()
    for distance in sorted_distances:
        if weight.get(distance[1]) and distance[0] != 0:
            weight[distance[1]] = weight[distance[1]] + (1 / distance[0])
        elif (not weight.get(distance[1])) and distance[0] != 0:
            weight[distance[1]] = (1 / distance[0])
        elif distance[0] == 0:
            weight[distance[1]] = 0
    
    # returns the most weighted type
    return max(weight, key=weight.get)

In [ ]:
# calculates the accuracy metric according to given formula
def accuracy_metric(original, predicted):
    correct = 0
    # compares predicted and original types
    for i in range(len(original)):
        if original[i] == predicted[i]:
            correct += 1
    return correct / float(len(original)) * 100.0

### PART 2: Concrete Material Strength Estimation from Data

In [39]:
# this function is the main function for regression
def regression():
    num_neighbors = [1, 3, 5, 7, 9]
    k_folds = 5
    data = read_csv_file("Concrete_Data_Yeh.csv")
    target = "csMPa"
    normalized_data = normalize_dataset(data, target)
    
    # invokes KNN regression function by changing data and algorithm types

     # KNN regression without feature normalization
    for num in num_neighbors:
        scores = knn_regression(data, k_folds, num, "normal", target)
   
     # KNN regression with feature normalization
    for num in num_neighbors:
        scores = knn_regression(normalized_data, k_folds, num, "normal", target)
        
    # weighted KNN regression without feature normalization
    for num in num_neighbors:
        scores = knn_regression(data, k_folds, num, "weighted", target)

    # weighted KNN regression with feature normalization
    for num in num_neighbors:
        scores = knn_regression(normalized_data, k_folds, num, "weighted", target)


In [36]:
# cross validation, csMPa prediction and mean absolute error calculation are done here
def knn_regression(dataset, n_folds, k_neighbors, algo_type, target):
    # splits data into 5 folds
    folds = cross_validation_splitter(dataset, n_folds)
    results = list()

    # create test and train sets
    for i in range(len(folds)):
        train_set = deepcopy(folds)
        test_set = train_set.pop(i)
        # original -> keeps original types for accuracy metric comparison
        original = list()

        for row in folds[i].iterrows():
            original.append(row[1][target])

        # according to algorithm type, call prediction functions
        predictions = list()
        for row in test_set.iterrows():
            if algo_type == "normal":
                output = predict_csMPa(train_set, row, k_neighbors, target)
                predictions.append(output)
            elif algo_type == "weighted":
                output = predict_weighted_csMPa(train_set, row, k_neighbors, target)
                predictions.append(output)
        
        # calculate mean absolute error
        mea = mean_absolute_error(original, predictions)
        results.append(mea)
    return results

In [33]:
def predict_csMPa(train_data, test_row, k, target):
    distances = dict()

    for train in train_data:
        for train_row in train.iterrows():
            distances[calculate_euclidean_distance(train_row, test_row)] = train_row[1][target]
    sorted_distances = sorted(distances.items(), key=lambda x: x[0])[:k]
    # get the average of distances
    return reduce(lambda a, b: a + b, [item[1] for item in sorted_distances]) / len([item[1] for item in sorted_distances])

We are looping over each point of the test data set, to find the euclidian distance between the test point and train data points. Then we are sorting the distance and finding the nearest K neighbours. Since we have the target values of the nearest neighbours, we average of those values and equate the predicted target variable.

In [32]:
def predict_weighted_csMPa(train_data, test_row, k, target):
    distances = dict()

    for train in train_data:
        for train_row in train.iterrows():
            distances[calculate_euclidean_distance(train_row, test_row)] = train_row[1][target]
    sorted_distances = sorted(distances.items(), key=lambda x: x[0])[:k]
    
    # x -> calculates weighted distance, y -> calculates total distance 
    x = reduce(lambda a, b: a + b, [(1/item[0]*item[1]) if item[0] != 0 else (item[1]) for item in sorted_distances])
    y = reduce(lambda a, b: a + b, [(1/item[0]) if item[0] != 0 else (item[1]) for item in sorted_distances])

    return x/y

In weighted k-NN model, the algorithm will still look at all k nearest neighbors, but the closer neighbors will have more of a vote than those further away.  The "predict_weighted_csMPa" function predicts the target variable "csMPa" of the query point using distance-weighted voting.

In [28]:
def mean_absolute_error(original, predicted):
    error = 0
    for i in range(len(original)):
        error += abs(original[i] - predicted[i])
    return error / len(original)

The "mean_absolute_error" function allows us to measure the accuracy of the model by comparing the actual values ​​and the predicted values. The returned value tells us that the average difference between the actual data value and the value predicted by the model.

### ANALYSIS FOR REGRESSION